In [1]:


from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.chat_models import ChatOllama

from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser


# # Create embeddingsclear


# # Create Ollama language model - Gemma 2


# Create the RAG chain using LCEL with prompt printing and streaming output

# Function to ask questions
embeddings = OllamaEmbeddings(model="nomic-embed-text", show_progress=True)



In [2]:
from dotenv import load_dotenv
import os

# Load the environment variables from the .env file
load_dotenv()

# Access the variables using os.getenv
GROQ_API_KEY = os.getenv("GROQ_API_KEY")


In [3]:

db = Chroma(persist_directory="./genese",
            embedding_function=embeddings)

# # Create retriever
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs= {"k": 5}
)

/home/meir/Documents/dev/.venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(


In [4]:
# local_llm = 'gemma2'
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-8b-8192")

# llm = ChatOllama(model=local_llm,
#                  keep_alive="3h", 
#                  max_tokens=512,  
#                  temperature=0)

# Create prompt template
template = """<bos><start_of_turn>user\nAnswer the question based only on the following context and extract out a meaningful answer. \
Please write in full sentences with correct spelling and punctuation. if it makes sense use lists. \
If the context doen't contain the answer, just respond that you are unable to find an answer. \
Use the question language to respond. \

CONTEXT: {context}

QUESTION: {question}

<end_of_turn>
<start_of_turn>model\n
ANSWER:"""
prompt = ChatPromptTemplate.from_template(template)

In [5]:
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
)


In [6]:
def ask_question(question):
    print("Answer:\n\n", end=" ", flush=True)
    for chunk in rag_chain.stream(question):
        print(chunk.content, end="", flush=True)
    print("\n")

# Example usage
if __name__ == "__main__":
    while True:
        user_question = input("Ask a question (or type 'quit' to exit): ")
        if user_question.lower() == 'quit':
            break
        answer = ask_question(user_question)
        # print("\nFull answer received.\n")

Answer:

 

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  6.58it/s]


Based on the provided context, it is clear that Sichem, the son of Hamor, had an illicit relationship with Dinah, the daughter of Jacob.

The relevant passages that support this answer are:

* "34.2 Son coeur s'attacha à Dina, fille de Jacob; il aima la jeune fille, et sut parler à son coeur." (This passage indicates that Sichem had fallen in love with Dinah and was thinking about her.)
* "34.4 Et Sichem dit à Hamor, son père: Donne-moi cette jeune fille pour femme." (This passage shows that Sichem wanted to marry Dinah.)
* "34.7 Et les fils de Jacob revenaient des champs, lorsqu'ils apprirent la chose; ces hommes furent irrités et se mirent dans une grande colère, parce que Sichem avait commis une infamie en Israël, en couchant avec la fille de Jacob, ce qui n'aurait pas dû se faire." (This passage reveals that Sichem had committed an egregious sin by having an illicit relationship with Dinah.)

Therefore, the answer to the question is that Sichem had an illicit relationship with Dina

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 17.08it/s]


Based on the provided context, I found the following information relevant to the question:

The context consists of biblical passages from the book of Genesis, specifically chapters 25, 9, 10, 28, 46, and 10.19.

Unfortunately, I was unable to find a direct answer to the question in the provided context. The context primarily discusses the genealogy of various biblical figures, including Ismaël, Isaac, Noé, and Jacob, as well as their descendants. It does not contain information that would directly answer the question.

If you could provide more context or clarify the question, I would be happy to try and assist you further.